<a href="https://colab.research.google.com/github/carolineoliveira994/100-days-of-code-python/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print("Diretório atual:", os.getcwd())


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric
from sklearn.metrics import f1_score


In [ ]:
!pip install transformers datasets scikit-learn


In [ ]:
pip install python-dotenv


In [ ]:
!pip install evaluate


In [ ]:
!pip install datasets


In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_sm


In [ ]:
!pip install transformers torch


In [ ]:
from google.colab import files
#uploaded = files.upload()

In [ ]:
print("Arquivos no diretório atual:", os.listdir())


In [ ]:
with open('.env', 'w') as f:
    f.write("""
BLUESKY_APP_USER = 'cocorolini.bsky.social'
BLUESKY_APP_PASS='3ovj-gbnh-trwd-k66r'

""")

In [ ]:
!python main.py search AQUECIMENTOGLOBAL --sort latest --limit 90


In [ ]:
from google.colab import files
import pandas as pd
# Defina o caminho correto para o arquivo CSV
file_path = '/content/data/search_results_AQUECIMENTOGLOBAL_2024_09_16.csv'
df = pd.read_csv(file_path)

df.head()


In [ ]:
import spacy

# Carregar o modelo de língua portuguesa
nlp = spacy.load('pt_core_news_sm')

# Função de pré-processamento usando spaCy
def preprocess_text_spacy(text_at):
    # Aplicar o modelo spaCy ao texto
    doc = nlp(text_at)
    # Extrair palavras lematizadas e remover stopwords e pontuação
    words = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(words)

# Aplicar pré-processamento ao DataFrame
df['text_at'] = df['text'].apply(preprocess_text_spacy)  # Substitua 'post_content' pelo nome real da coluna


In [ ]:
print(df[['text', 'text_at']].head())


In [ ]:
df = pd.read_csv('/content/data/search_results_AQUECIMENTOGLOBAL_2024_09_16.csv')


In [ ]:
df

In [ ]:
pd.set_option('display.max_columns', None)
print(df)

In [ ]:
import pandas as pd


In [ ]:
df.columns

In [ ]:
columns_to_remove = ["reply_count", "author_display_name", "author_handle", "indexed_at", "cid", "uri", "created_at", "repost_count", "like_count"]
df = df.drop(columns=columns_to_remove)

# Exibir DataFrame após remover colunas
print("\nDepois:")
print(df)

In [ ]:
df

In [ ]:
labels =
[0,0,0,1,1,0,0,0,0,
0,0,0,0,0,0,0,0,0,0,
0,0,0,0,0,0,0,0,0,0,
1,0,1,0,0,0,0,0,0,1,
0,0,0,0,0,0,0,0,1,1,
1,0,0,0,1,0,0,0,0,0,
0,0,0,0,0,0,0,0,0,0,
0,0,1,1,0,0,0,0,1,0,
1,0,0,0,0,0,0,0,0]


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
import numpy as np
import pandas as pd

# Tokenização
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Supondo que você já tenha um DataFrame `df` com as colunas 'text' e 'label'
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=128  # Defina um tamanho máximo para padding e truncamento
    )

# Supondo um DataFrame com mais amostras
df = pd.DataFrame({
    'text': ["Seu texto aqui"] * 100,  # Adicione textos suficientes
    'label': [0] * 100  # Adicione rótulos suficientes
})

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Dividindo os dados em treino e validação
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# Carregar métricas
metric = evaluate.load('f1')

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return metric.compute(predictions=preds, references=labels)

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    remove_unused_columns=False  # Adicionando esta linha para evitar erros de coluna
)

# Inicializar o modelo BERT
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)

# Treinamento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
